<a href="https://colab.research.google.com/github/huangchengwu/NLP_bert/blob/master/faiss_text_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0


In [2]:
!pip install faiss-cpu sentence-transformers





In [3]:
!pip install --force-reinstall numpy==1.26.4





  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [4]:
import torch
print(torch.cuda.is_available())  # 如果返回 True，说明有可用的 GPU


True


In [9]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "什么是向量数据库？"},
]
pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
pipe(messages)

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': '什么是向量数据库？'},
   {'role': 'assistant', 'content': '嗯，向量数据库是什么？我以前听说过数据库，就是用来存储数据的结构，但'}]}]

In [10]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# 定义并加载嵌入模型
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 动态获取实际嵌入维度
sample_embedding = model.encode(["测试"])[0]
dim = len(sample_embedding)

# 初始化 FAISS 向量数据库
index = faiss.IndexFlatL2(dim)

# 示例文档列表
documents = [
    "FAISS 是一个高效的向量数据库，专门用于近似最近邻搜索。",
    "SentenceTransformers 是一个用于生成文本嵌入的库，适合用在语义搜索中。",
    "向量数据库可以用于存储大规模的文本数据，并且可以高效地进行相似性搜索。",
    "MySQL 是一个关系型数据库管理系统，可以存储和查询结构化数据。",
    "机器学习可以用于文本分类、回归、聚类等任务。"
]

# 将文档嵌入并添加到 FAISS 向量数据库
def embed_and_upsert_document(index, document_id, document_content):
    embedding = model.encode([document_content])[0]
    embedding = np.array(embedding, dtype=np.float32).reshape(1, -1)
    index.add(embedding)
    print(f"Document {document_id} added to FAISS.")

# 添加文档
for idx, doc in enumerate(documents):
    embed_and_upsert_document(index, idx, doc)

print("All documents added to FAISS.")

# 查询 FAISS
def query_faiss(index, query_text, n_results=3):
    query_embedding = model.encode([query_text])[0]
    query_embedding = np.array(query_embedding, dtype=np.float32).reshape(1, -1)
    distances, indices = index.search(query_embedding, n_results)
    return distances, indices
query_text = "什么是向量数据库？"


# 示例查询                                                                                                                                                                                                                                                                                            = "什么是向量数据库？"
distances, indices = query_faiss(index, query_text)

print("==== FAISS 查询结果 ====")
for i in range(len(indices[0])):
    doc_idx = indices[0][i]
    print(f"最相关文档 {i+1}: {documents[doc_idx]} (距离: {distances[0][i]:.4f})")

pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
pipe(messages)


# 拼接检索结果作为上下文
retrieved_context = "\n".join([documents[i] for i in indices[0]])

# 构建提示词（prompt）
prompt = f"""请根据以下背景知识回答问题：
{retrieved_context}

问题：{query_text}
回答："""

print("prompt",prompt)
# 生成回答
response = pipe(prompt, max_new_tokens=200, do_sample=False)
print(response[0]['generated_text'])



Document 0 added to FAISS.
Document 1 added to FAISS.
Document 2 added to FAISS.
Document 3 added to FAISS.
Document 4 added to FAISS.
All documents added to FAISS.
==== FAISS 查询结果 ====
最相关文档 1: 向量数据库可以用于存储大规模的文本数据，并且可以高效地进行相似性搜索。 (距离: 17.2017)
最相关文档 2: 机器学习可以用于文本分类、回归、聚类等任务。 (距离: 24.0482)
最相关文档 3: MySQL 是一个关系型数据库管理系统，可以存储和查询结构化数据。 (距离: 32.0252)


Device set to use cuda:0


prompt 请根据以下背景知识回答问题：
向量数据库可以用于存储大规模的文本数据，并且可以高效地进行相似性搜索。
机器学习可以用于文本分类、回归、聚类等任务。
MySQL 是一个关系型数据库管理系统，可以存储和查询结构化数据。

问题：什么是向量数据库？
回答：
请根据以下背景知识回答问题：
向量数据库可以用于存储大规模的文本数据，并且可以高效地进行相似性搜索。
机器学习可以用于文本分类、回归、聚类等任务。
MySQL 是一个关系型数据库管理系统，可以存储和查询结构化数据。

问题：什么是向量数据库？
回答：向量数据库是存储结构化的文本数据，通过向量表示方法将文本转化为向量，从而实现相似性搜索。向量表示方法包括向量空间模型、余弦相似度、向量长度、向量夹角余弦等。向量数据库支持向量查询，通过向量的长度、夹角余弦等进行检索。向量数据库支持向量索引，通过向量的长度、夹角余弦等进行索引优化。向量数据库支持向量搜索，通过向量的长度、夹角余弦等进行搜索。向量数据库支持向量聚类，通过向量的长度、夹角余弦等进行聚类。向量数据库支持向量分类，通过向量的长度、夹角余弦等进行分类。向量数据库支持向量回归，通过向量的长度、夹角余弦等进行回归。向量数据库支持向量聚类，
